## construction phase

In [1]:
import tensorflow as tf

In [2]:
n_inputs=28*28  #mnist
n_hidden1=300
n_hidden2=100
n_outputs=10

In [5]:
## we dont know thw size of the training batch so shape is (None).
X=tf.placeholder(tf.float32,shape=(None,n_inputs),name="X")
y=tf.placeholder(tf.int64,shape=(None),name="y")

In [12]:
## smaller code of the folowing code is available in sk-learn see in the book
def neuron_layer(X,n_neurons,name,activation=None):
    with tf.name_scope(name):
        n_inputs =int(X.get_shape()[1])
        stddev=2/np.sqrt(n_inputs)
        init=tf.truncated_normal((n_inputs,n_neurons),stddev=stddev)
        W=tf.Variable(init,name="kernel")
        b=tf.Variable(tf.zeros([n_neurons]),name="bias")
        Z=tf.matmul(X,W)+b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [13]:
## now lets create deep neural network
import numpy as np
with tf.name_scope("dnn"):
    hidden1=neuron_layer(X,n_hidden1,name="hidden1",activation=tf.nn.relu)
    hidden2=neuron_layer(hidden1,n_hidden2,name="hidden2",activation=tf.nn.relu)
    logits=neuron_layer(hidden2,n_outputs,name="hidden2")
    

In [15]:
## lets compute our lcost function using cross entropy
with tf.name_scope("loss"):
    xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss=tf.reduce_mean(xentropy,name="loss")

In [17]:
## lets reduce our cost function using gradient descent
learning_rate=0.01
with tf.name_scope("train"):
    optimizer=tf.train.GradientDescentOptimizer(learning_rate)
    training_op=optimizer.minimize(loss)

In [18]:
## following code will give us the accuracy
with tf.name_scope("eval"):
    correct=tf.nn.in_top_k(logits,y,1)
    accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))

In [19]:
## lets initialze and save our model
init=tf.global_variables_initializer()
saver=tf.train.Saver()

## Execution phase

In [20]:
# lets load the data
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [25]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

11493376/11490434 [==============================] - 3s 0us/step


In [26]:
# lets define number of epochs we want to run and size of mini_batches
n_epochs=40
batch_size=50

In [27]:
#shuffling the mini batches
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch


In [28]:
# training the model
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Val accuracy:", acc_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Batch accuracy: 0.96 Val accuracy: 0.9114
1 Batch accuracy: 0.98 Val accuracy: 0.9304
2 Batch accuracy: 0.98 Val accuracy: 0.9368
3 Batch accuracy: 0.92 Val accuracy: 0.9438
4 Batch accuracy: 0.96 Val accuracy: 0.9492
5 Batch accuracy: 0.94 Val accuracy: 0.9516
6 Batch accuracy: 0.96 Val accuracy: 0.9558
7 Batch accuracy: 0.96 Val accuracy: 0.961
8 Batch accuracy: 1.0 Val accuracy: 0.9624
9 Batch accuracy: 0.98 Val accuracy: 0.9628
10 Batch accuracy: 0.98 Val accuracy: 0.9658
11 Batch accuracy: 0.96 Val accuracy: 0.9664
12 Batch accuracy: 0.96 Val accuracy: 0.969
13 Batch accuracy: 1.0 Val accuracy: 0.969
14 Batch accuracy: 1.0 Val accuracy: 0.97
15 Batch accuracy: 1.0 Val accuracy: 0.972
16 Batch accuracy: 0.98 Val accuracy: 0.9734
17 Batch accuracy: 1.0 Val accuracy: 0.973
18 Batch accuracy: 0.98 Val accuracy: 0.9734
19 Batch accuracy: 1.0 Val accuracy: 0.9734
20 Batch accuracy: 1.0 Val accuracy: 0.9744
21 Batch accuracy: 0.98 Val accuracy: 0.9758
22 Batch accuracy: 0.98 Val accura

In [29]:
## lets make prediction 
## but before this chage the execution phase like following:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
    X_new_scaled = X_test[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [32]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", y_test[:20])

Predicted classes: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
Actual classes:    [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
